## Chat Bot Evaluation as Multi-agent Simulation

When building a chat bot, such as a customer support assistant, it can be hard to properly evalute your bot's performance. It's time-consuming to have to manually interact with it intensively for each code change.

One way to make the evaluation process easier and more reproducible is to simulate a user interaction.

With LangGraph, it's easy to set this up. Below is an example of how to create a "virtual user" to simulate a conversation.

The overall simulation looks something like this:

![diagram](./img/virtual_user_diagram.png)

**First,** we'll set up our environment.

In [1]:
# %%capture --no-stderr
# %pip install -U langgraph langchain langchain_openai

In [2]:
import getpass
import os
import uuid


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")

# Optional, add tracing in LangSmith.
# This will help you visualize and debug the control flow
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agent Simulation Evaluation"

## 1. Your Agent API

For this notebook, we assume your agent's API accepts a list of messages and responds with a message.
This is configurable and can run on another system (e.g., if your system isn't running in python).

In [3]:
from typing import List

import openai


# This is flexible, but you can define your agent here, or call your agent API here.
def my_chat_bot(messages: List[dict]) -> dict:
    completion = openai.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo"
    )
    return completion.choices[0].message.model_dump()

## 2. Define the Agent Simulation

In [12]:
import operator
from typing import Annotated, Callable, Dict, List, TypedDict

from langchain.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph

SIMULATED_USER_NAME = "simulated"


# This is just an example, we can
# configure additional parameters if
# you want more control
class SimulatedUserConfig(TypedDict):
    system_prompt: str


# This is the input to every node in the simulation graph
# It tracks the graph state over time. Our only "state"
# is the conversation messages, while the user config
# is provided to make the virtual user more unique or realistic
class Environment(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    simulated_user_config: SimulatedUserConfig


# We currently let the virtual user decide if the conversation can end
# we could also track max conversation turns, add a conversation "supervisor"
# or use other heuristics to control the dialogue flow
def should_continue(state: Environment):
    """Determine if the simulation should continue."""
    if state["messages"][-1].content.strip().endswith("FINISHED"):
        return "end"
    return "continue"


## The next two functions define the API between the simulation
# and the chat bot you wish to test.
# We are assuming your chat bot accepts a list of OAI messages
@chain
def get_messages_for_agent(state: Environment):
    """Convert the simulation state to the input

    for your agent you want to evaluate."""
    return [convert_message_to_dict(message) for message in state["messages"]]


# This takes the output of your chat bot
# and adds it to the simulation state
def get_response_message_from_agent(agent_output):
    """Get the response from the agent you are evaluting,
    and use it to update the simulation state."""
    # If we do an ai message here, the user proxy llm
    # will usually forget it's acting.
    return {"messages": [HumanMessage(content=agent_output["content"])]}


# This is run once at the beginning of the simulation.
# It's more convenient to just write an input string
# than to pass in a full message, but this could be removed below
def enter(inputs: dict):
    """Start the simulation. This makes it less verbose to invoke."""
    inputs["messages"] = [
        HumanMessage(content=inputs["input"], name=SIMULATED_USER_NAME)
    ]
    return inputs


def create_simulation(chat_bot: Callable[[List[Dict]], Dict], simulated_user_llm=None):
    """Create a chat bot simulation graph.

    Args:
        - chat_bot: the agent you are evaluating. Accepts a list of openai messages
            and returns an openai assistant message
        - simulated_user_llm: the LLM to power your virtual user.
            Defaults to gpt-4-1106-preview
    Returns:
        - simulation: an runnable object formed from compiling the state graph
    """
    # This defines the virtual user proxy
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are role-playing a human character: '{name}'. "
                "You are not an AI assistant and you are not supposed to help or assist."
                " You must behave as this human would throughout the conversation below.\n\n"
                "Your messages will bear the name 'simulated', but DO NOT under any circumstances"
                "say that you are 'simulated'. You will be evaluated based on how realistic your"
                "impersonation of this character is.  This must feel real! Here are the details for your character:"
                "\n"
                "{system_prompt}"  # This is the value you provide to characterize the user
                '\n\nWhen you are finished with the conversation, respond with a single word "FINISHED"',
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    ).partial(name=SIMULATED_USER_NAME)
    simulated_user_llm = simulated_user_llm or ChatOpenAI(model="gpt-4-1106-preview")
    user_proxy = (
        (lambda x: {**x, **x["simulated_user_config"]})
        | prompt
        | simulated_user_llm
        | (
            lambda x: {
                "messages": [HumanMessage(content=x.content, name=SIMULATED_USER_NAME)]
            }
        )
    )
    graph_builder = StateGraph(Environment)
    graph_builder.add_node("user", user_proxy)
    graph_builder.add_node(
        # The "|" syntax composes these steps in the pipeline to map between
        # the simulation state and your chat bot's API
        "chat_bot",
        get_messages_for_agent | chat_bot | get_response_message_from_agent,
    )
    # Every response from  your chat bot will automatically go to the
    # simulated user
    graph_builder.add_edge("chat_bot", "user")
    graph_builder.add_conditional_edges(
        "user",
        should_continue,
        # If the finish criteria are met, we will stop the simulation,
        # otherwise, the virtual user's message will be sent to your chat bot
        {
            "end": END,
            "continue": "chat_bot",
        },
    )
    # The input will first go to your chat bot
    graph_builder.set_entry_point("chat_bot")
    return (enter | graph_builder.compile()).with_config(run_name="Agent Simulation")

## Run Simulation

Now we can evaluate our chat bot!

In [13]:
simulation = create_simulation(my_chat_bot)

In [14]:
from langchain_core.tracers.context import tracing_v2_enabled

# The tracing context manager lets us easily fetch the trace URL in-context.
# You can turn this off if you don't want to trace the execution.
with tracing_v2_enabled() as tracer:
    result = simulation.invoke(
        {
            "simulated_user_config": {
                "system_prompt": "You are on a budget. Your family is hard to please."
                " They all like the beach, except for Aunt Lily, who prefers the mountains."
            },
            "input": "help me plan my family vacation",
        }
    )
    # You can go to this run to review the entire simulation trace
    url = tracer.get_run_url()

Skipping write for channel input which has no readers


### Review Result

If you've traced the run, you can see the resulting trace in the UI by clicking on the url.
Select the last 'ChatOpenAI' call in the trace to see the full conversation in a single view.

![full-conversation](./img/virtual_user_full_convo.png)


From this run, you can manually annotate it to score its quality.

![annotate](./img/virtual_user_annotate.png)

In [ ]:
url

In [10]:
url

'https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/191a0aa8-051e-41e6-91f8-1a213087e358/r/d3f6e590-bae8-402e-83a7-46739279c061?poll=true'

In [11]:
result["messages"]

[HumanMessage(content='help me plan my family vacation', name='simulated'),
 HumanMessage(content="Of course! I'd be happy to help you plan your family vacation. Please provide me with some additional information about your preferences, such as your desired destination, budget, travel dates, and any specific activities or interests you would like to incorporate into your trip."),
 HumanMessage(content='simulated', name='simulated'),
 HumanMessage(content="Sure! Even though I cannot physically make bookings or arrangements, I can certainly help you with the planning process and give you some suggestions. Please let me know the following details:\n\n1. Destination: Where would you like to go? Do you have a specific country, city, or region in mind?\n\n2. Duration: How long are you planning to go on vacation? Do you have specific dates in mind?\n\n3. Budget: Have you determined a budget for your trip? Knowing your budget will help me provide recommendations within your means.\n\n4. Intere

In [ ]:
result = simulation.invoke(
    {
        "simulated_user_config": {"system_prompt": "You are "},
        "input": "help me plan my family vacation",
    }
)